In [ ]:
!pip install --upgrade yfinance

In [ ]:
!pip install pandas jpholiday

In [ ]:
import pandas as pd
from google.colab import drive

# Google Driveをマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import time

# ファイルパスを指定
file_path = '/content/drive/MyDrive/csvファイル保存用/seikei_merged_df2_v3.csv'

# CSVファイルを読み込む
df_Y = pd.read_csv(file_path)

# データフレームの先頭10行を取得
#df_Y = df.head(100).copy()  # スライスのコピーを作成


# 証券コードを整数型に変換
#df_Y["証券コード"] = df_Y["証券コード"].astype(int)

# 証券コードが4448の最初の行のインデックスを取得
start_index = df_Y[df_Y["証券コード"] == 4448].index[0]

# その行以降の1000行を取得
df_Y = df_Y.iloc[start_index : start_index + 200].copy()

# データフレームの先頭5行を表示
print(df_Y.head())
df_Y.shape

    証券コード         提出日   提出日_180日後
81   4448  2023-03-30  2023-09-26
82   4180  2023-03-30  2023-09-26
83   5027  2023-03-30  2023-09-26
84   2170  2023-03-30  2023-09-26
85   6915  2023-03-30  2023-09-26


(200, 3)

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import jpholiday
from tqdm import tqdm
from IPython.display import display
import time

# 土日や祝日をスキップして次の平日を返す関数
def adjust_to_next_weekday(date):
    if isinstance(date, str):  # 文字列ならdatetimeに変換
        date = datetime.strptime(date, "%Y-%m-%d")
    while date.weekday() >= 5 or jpholiday.is_holiday(date):  # 土日または祝日なら
        date += timedelta(days=1)  # 次の平日に進む
    return date

def get_adjusted_close_with_calculation(code, date):
    """
    指定された証券コードと日付について、調整済み終値を取得する。
    Adjusted Closeがある場合はそのまま、ない場合はCloseを分割調整。
    """
    ticker = f"{code}.T"
    start_date = date.strftime("%Y-%m-%d")
    end_date = (date + timedelta(days=1)).strftime("%Y-%m-%d")

    ticker_data = yf.Ticker(ticker)

    hist = ticker_data.history(start=start_date, end=end_date)
    splits = ticker_data.splits

    if not hist.empty:
      if 'Adj Close' in hist.columns:  # Adjusted Closeがある場合
          return hist["Adj Close"].iloc[0]
      elif 'Close' in hist.columns:  # Adjusted Closeがない場合
          close_price = hist["Close"].iloc[0]

          # 分割調整
          if not splits.empty:
              try:
                  split_factor = splits[splits.index <= date].prod()  # 累積分割比率
                  if split_factor != 0:
                      close_price /= split_factor  # 分割後価格で調整
              except Exception as e:
                  print(f"Error calculating split adjustment: {e}")
                  # 必要に応じてデフォルト値を返すなどの処理を追加

          return close_price
      else:
        raise ValueError("Historical data is empty.")
        return None

def get_adjusted_close_for_month(code, start_date, max_failures=5):
    """
    提出日から1ヶ月分の調整済み終値を取得する。
    Adjusted CloseがなければCloseを分割調整して取得。
    max_failures: 最大失敗回数を設定
    """
    adjusted_closes = []
    current_date = start_date
    total_days = 0  # 実際に取得した日数をカウントする
    failure_count = 0  # 失敗回数をカウント

    # 6ヶ月分以上のデータを取得するまで繰り返し取得
    while total_days < 180:
        adjusted_date = adjust_to_next_weekday(current_date)

        # 同一の日付の調整済み終値を取得しないように、重複チェックを追加
        if any(adjusted_date == item['日付'] for item in adjusted_closes):
            current_date = adjust_to_next_weekday(current_date + timedelta(days=1))  # 重複している場合は次の日に進む
            continue

        adj_close_price = get_adjusted_close_with_calculation(code, adjusted_date)

        if adj_close_price is not None:
            adjusted_closes.append({
                "証券コード": code,
                "日付": adjusted_date,
                "調整済み終値": adj_close_price
            })
            total_days += 1  # 取得したデータのカウントを進める
            failure_count = 0  # 成功した場合、失敗カウントをリセット
            current_date += timedelta(days=1) # データを取得した日を次の日に進める
        else:
            failure_count += 1
            if failure_count >= max_failures:
                print(f"銘柄 {code} は失敗回数が {max_failures} 回に達したため、処理をスキップします。")
                break  # 最大失敗回数に達したら処理をスキップ
            current_date += timedelta(days=1)  # 失敗した場合は次の日に進む

        current_date = adjust_to_next_weekday(current_date + timedelta(days=1))

    return adjusted_closes

def process_in_batches(df, batch_size):
    """
    バッチ処理を行う関数。
    """
    adjusted_data = []
    num_batches = (len(df) + batch_size - 1) // batch_size

    for batch_num in range(num_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(df))
        batch_df = df.iloc[start_idx:end_idx]

        # tqdmで進捗バーを表示
        for index, row in tqdm(batch_df.iterrows(), total=batch_df.shape[0], desc=f"バッチ {batch_num + 1}/{num_batches}"):
            code = row["証券コード"]

            # 提出日_180日後をdatetime型に変換してから1日追加
            start_date = datetime.strptime(row["提出日"], "%Y-%m-%d") + timedelta(days=1)

            adjusted_closes = get_adjusted_close_for_month(code, start_date)

            # 調整済み終値のデータを追加
            adjusted_data.extend(adjusted_closes)

            # リクエストの間隔を調整（1秒待機）
            time.sleep(5)

    return adjusted_data

# バッチ処理を使用して調整済み終値を取得
batch_size = 100  # バッチサイズを設定

adjusted_data = process_in_batches(df_Y, batch_size=batch_size)

# 新しいデータフレームに結果を格納
df_adjusted = pd.DataFrame(adjusted_data)
print(df_adjusted.columns)

# 銘柄ごとのデータ数をカウントし、180未満のものを削除
df_adjusted = df_adjusted.groupby('証券コード').filter(lambda x: len(x) >= 180)

# 結果を表示
display(df_adjusted)


バッチ 1/2:   0%|          | 0/100 [00:00<?, ?it/s]ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 2023-03-31 -> 2023-04-01)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 1926-02-08 -> 2025-01-14)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 2023-04-03 -> 2023-04-04)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 1926-02-08 -> 2025-01-14)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 2023-04-05 -> 2023-04-06)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 1926-02-08 -> 2025-01-14)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 2023-04-07 -> 2023-04-08)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 1926-02-08 -> 2025-01-14)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 2023-04-10 -> 2023-04-11)
ERROR:yfinance:$4448.T: possibly delisted; no price data found  (1d 1926-02-08 -> 2025-01-14)


銘柄 4448 は失敗回数が 5 回に達したため、処理をスキップします。


バッチ 1/2:   0%|          | 0/100 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import os

# Google Drive 内の保存先パスを指定
base_path = '/content/drive/My Drive/yfinance_csvfile/df_savedyfinance'
file_extension = '.csv'

# ファイル番号を自動的に増加させて新しいファイル名を決定
file_number = 1
save_path = f"{base_path}{file_number}{file_extension}"

# 同じ名前のファイルがすでに存在するか確認し、存在する場合はファイル番号を増やす
while os.path.exists(save_path):
    file_number += 1
    save_path = f"{base_path}{file_number}{file_extension}"

# CSV ファイルとして保存
df_adjusted.to_csv(save_path, index=False, encoding='utf-8-sig')  # `utf-8-sig` で日本語対応

print(f"DataFrame を保存しました: {save_path}")


DataFrame を保存しました: /content/drive/My Drive/yfinance_csvfile/df_savedyfinance13.csv
